<a href="https://colab.research.google.com/github/cbalkig/Anomaly_Detection_in_Videos/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import keras
from keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import time

In [ ]:
working_directory = '/content/drive/MyDrive/AnomalyDetectionInVideos'

In [ ]:
class Config:
    EDIT_DATASET_PATH = os.path.join(working_directory, "regenerated_files")
    BATCH_SIZE = 256
    EPOCHS = 10
    MODEL_PATH = os.path.join(working_directory, "model.hdf5")
    IMAGE_SIZE = 256

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(10):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n, :, :, 0], cmap='gray')
        plt.axis('off')

In [ ]:
def plot_acc_loss(trained):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(trained.epoch, trained.history["loss"], label="Train loss")
    ax[1].set_title('acc')
    ax[1].plot(trained.epoch, trained.history["accuracy"], label="Train acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:
def train_model(reload_model=True):
    if reload_model:
        return load_model(Config.MODEL_PATH)
    input_shape = (Config.IMAGE_SIZE, Config.IMAGE_SIZE, 1)

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))

    model.summary()

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=Config.EPOCHS, verbose = 1)
    plot_acc_loss(history)
    return model

In [ ]:
start_time = time.time()
print("Sample Count", sum(len(files) for _, _, files in os.walk(Config.EDIT_DATASET_PATH)))
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=30, 
        fill_mode='nearest',
        brightness_range=[0.4,1.5])
train_generator = train_datagen.flow_from_directory(
        Config.EDIT_DATASET_PATH,
        target_size=(Config.IMAGE_SIZE, Config.IMAGE_SIZE),
        batch_size=Config.BATCH_SIZE,
        shuffle=True,
        color_mode = 'grayscale',
        class_mode='binary')
print("Sample Count after ImageDataGenerator", train_generator.samples)

image_batch, label_batch = next(train_generator)
show_batch(image_batch, label_batch)
print("ImageDataGenerator Execution time:", time.time() - start_time, "seconds.")

In [ ]:
start_time = time.time()
model = train_model(reload_model=False)
print("Train Execution time:", time.time() - start_time, "seconds.")
print("Model is ready.")
model.save(Config.MODEL_PATH)
print("Model saved : ", Config.MODEL_PATH)